# Description du projet : 

Contexte :  L'INSEE est l'institut officiel français qui collecte des données de tous types sur le territoire français. Elles peuvent être démographiques (Naissances, Décès, Densité de la population...), économiques (Salaires, Entreprises par activité / taille...) et plus encore.
Ces données peuvent être d'une grande aide pour observer et mesurer les inégalités au sein de la population française.

## Objectif : Comparer les inégalités en France : 

* Entreprises en fonction de leur localisation, de leur taille. 
* Population en fonction du salaire et de la localisation.
* Focus sur une grande ville 

## Ressources à consulter : 
* Données : https://assets-datascientest.s3.eu-west-1.amazonaws.com/notebooks/power_bi/power_bi_datasets_projet/french_industry.zip


### Informations sur le nombre d'entreprises dans chaque ville française classées par taille.
<u>base_etablissement_par_tranche_effectif.csv :</u>

* CODGEO : ID géographique de la ville
* LIBGEO : nom de la ville
* REG : numéro de région
* DEP : numéro de département
* E14TST : nombre total d'entreprises dans la ville
* E14TS0ND : nombre d'entreprises de taille inconnue ou nulle dans la ville
* E14TS1 : nombre d'entreprises de 1 à 5 employés dans la ville
* E14TS6 : nombre d'entreprises de 6 à 9 employés dans la ville
* E14TS10 : nombre d'entreprises de 10 à 19 employés dans la ville
* E14TS20 : nombre d'entreprises de 20 à 49 employés dans la ville
* E14TS50 : nombre d'entreprises de 50 à 99 employés dans la ville
* E14TS100 :  nombre d'entreprises de 100 à 199 employés dans la ville
* E14TS200 : nombre d'entreprises de 200 à 499 employés dans la ville
* E14TS500 : nombre d'entreprises de plus de 500 employés dans la ville

In [1]:
# Suppression des warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
# Lecture fichier csv
df1 = pd.read_csv('base_etablissement_par_tranche_effectif.csv',dtype={'REG':'str'})
# Tous les nom de ville mis à la première lettre en masjuscule
df1.LIBGEO = df1.LIBGEO.str.title()
# Suppression des accents les plus communs
df1['LIBGEO'] = df1['LIBGEO'].str.replace("É","E")
df1['LIBGEO'] = df1['LIBGEO'].str.replace("Î","I")
#correction code geo erroné commune "L'Oudon"
df_temp = df1.loc[df1.CODGEO=='14697']
df_temp.CODGEO = '14472'
df1.loc[df1.CODGEO=='14697']=df_temp
# Extraction des données GEO de la DF
df1_base = df1.iloc[:,0:4]
# Affichage des infos du dataset
df1_base.info()
df1_base.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36681 entries, 0 to 36680
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CODGEO  36681 non-null  object
 1   LIBGEO  36681 non-null  object
 2   REG     36681 non-null  object
 3   DEP     36681 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


,CODGEO,LIBGEO,REG,DEP
0,01001,L'Abergement-Clémenciat,82,01
1,01002,L'Abergement-De-Varey,82,01
2,01004,Ambérieu-En-Bugey,82,01
3,01005,Ambérieux-En-Dombes,82,01
4,01006,Ambléon,82,01


### Données géographiques sur les villes françaises (principalement la latitude et la longitude, mais aussi les codes et les noms des régions/départements).
<u>name_geographic_information.csv : </u>

* code_région
* nom_région
* chef.lieu_région
* numéro_département
* nom_département
* préfecture
* numéro_circonscription
* nom_commune
* codes_postaux
* code_insee
* latitude
* longitude
* éloignement

In [3]:
# Utilisation du fichier données GPS (étude Dominique)
geo = pd.read_csv('geo_2014.csv',usecols=['COM','Code_postal','latitude','longitude'])
geo = geo.drop_duplicates(subset='COM',keep='first')
geo

,COM,Code_postal,latitude,longitude
0,01001,1400.0,46.153721,4.925850
1,01002,1640.0,46.009606,5.428088
2,01004,1500.0,45.961049,5.372275
3,01005,1330.0,45.996164,4.911967
4,01006,1300.0,45.749886,5.594585
...,...,...,...,...
43114,97613,97650.0,-12.751168,45.086393
43115,97614,97670.0,-12.837567,45.138321
43117,97615,97615.0,-12.796910,45.284254
43118,97616,97640.0,-12.861040,45.118132


In [4]:
# Lecture du fichier csv - insee source année 2014
df2_2014 = pd.read_excel('codes_geo_2014.xlsx',dtype='str')
df2_2014.info()
df2_2014.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36681 entries, 0 to 36680
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   CODGEO            36681 non-null  object
 1   REG               36681 non-null  object
 2   DEP               36681 non-null  object
 3   LIBGEO            36681 non-null  object
 4   nom_région        36681 non-null  object
 5   nom_département   36681 non-null  object
 6   chef.lieu_région  36681 non-null  object
dtypes: object(7)
memory usage: 2.0+ MB


,CODGEO,REG,DEP,LIBGEO,nom_région,nom_département,chef.lieu_région
0,01001,82,01,Abergement-Clémenciat,Rhône-Alpes,Ain,Bourg-en-Bresse
1,01002,82,01,Abergement-de-Varey,Rhône-Alpes,Ain,Bourg-en-Bresse
2,01004,82,01,Ambérieu-en-Bugey,Rhône-Alpes,Ain,Bourg-en-Bresse
3,01005,82,01,Ambérieux-en-Dombes,Rhône-Alpes,Ain,Bourg-en-Bresse
4,01006,82,01,Ambléon,Rhône-Alpes,Ain,Bourg-en-Bresse


In [5]:
df2 = pd.merge(df2_2014,geo,how='left',left_on='CODGEO',right_on='COM')

In [6]:
df2 = df2.drop_duplicates(subset='CODGEO',keep='first')
df2.loc[df2.duplicated(subset='CODGEO')]

,CODGEO,REG,DEP,LIBGEO,nom_région,nom_département,chef.lieu_région,COM,Code_postal,latitude,longitude


In [7]:
df2_base = df2.drop(labels=['REG','DEP','LIBGEO','COM'],axis='columns')

In [8]:
# Vérification des doublons
df2_base.loc[df2_base.duplicated()]

,CODGEO,nom_région,nom_département,chef.lieu_région,Code_postal,latitude,longitude


In [9]:
df2_base.isna().sum()

CODGEO                 0
nom_région             0
nom_département        0
chef.lieu_région       0
Code_postal         1739
latitude            1739
longitude           1739
dtype: int64

In [10]:
# Fusion des DataFrame "Etablissement par tranche d'effectifs" et "données géographiques"
all_content = pd.merge(df1_base,df2_base,how='left',on='CODGEO')
all_content.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36681 entries, 0 to 36680
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CODGEO            36681 non-null  object 
 1   LIBGEO            36681 non-null  object 
 2   REG               36681 non-null  object 
 3   DEP               36681 non-null  object 
 4   nom_région        36681 non-null  object 
 5   nom_département   36681 non-null  object 
 6   chef.lieu_région  36681 non-null  object 
 7   Code_postal       34942 non-null  float64
 8   latitude          34942 non-null  float64
 9   longitude         34942 non-null  float64
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [11]:
# Taille de la DataFrame
all_content.isna().sum()

CODGEO                 0
LIBGEO                 0
REG                    0
DEP                    0
nom_région             0
nom_département        0
chef.lieu_région       0
Code_postal         1739
latitude            1739
longitude           1739
dtype: int64

In [12]:
# Recherche des duplications
all_content_dupli = all_content.loc[all_content.duplicated(subset=['CODGEO','LIBGEO'])]
# Affichage des duplications
all_content_dupli

,CODGEO,LIBGEO,REG,DEP,nom_région,nom_département,chef.lieu_région,Code_postal,latitude,longitude


In [13]:
# Affichage de la dimension de la DF all_content
all_content.shape

(36681, 10)

### Salaires par villes française par catégories d'emploi, âge et sexe

<u>net_salary_per_town_categories.csv :</u> 

* CODGEO : ID géographique de la ville
* LIBGEO : nom de la ville
* SNHM14 : salaire net moyen
* SNHMC14 : salaire net moyen par heure pour les cadres
* SNHMP14 : salaire net moyen par heure pour un cadre moyen
* SNHME14 : salaire net moyen par heure pour l'employé
* SNHMO14 :  salaire net moyen par heure pour le travailleur
* SNHMF14 : salaire net moyen pour les femmes
* SNHMFC14 : salaire net moyen par heure pour les cadres féminins
* SNHMFP14 : salaire net moyen par heure pour les cadres moyens féminins
* SNHMFE14 : salaire net moyen par heure pour une employée 
* SNHMFO14 : salaire net moyen par heure pour une travailleuse 
* SNHMH14 : salaire net moyen pour un homme
* SNHMHC14 : salaire net moyen par heure pour un cadre masculin
* SNHMHP14 : salaire net moyen par heure pour les cadres moyens masculins
* SNHMHE14 : salaire net moyen par heure pour un employé masculin
* SNHMHO14 : salaire net moyen par heure pour un travailleur masculin
* SNHM1814 : salaire net moyen par heure pour les 18-25 ans
* SNHM2614 : salaire net moyen par heure pour les 26-50 ans
* SNHM5014 : salaire net moyen par heure pour les >50 ans
* SNHMF1814 : salaire net moyen par heure pour les femmes âgées de 18 à 25 ans
* SNHMF2614 : salaire net moyen par heure pour les femmes âgées de 26 à 50 ans
* SNHMF5014 : salaire net moyen par heure pour les femmes de plus de 50 ans
* SNHMH1814 : salaire net moyen par heure pour les hommes âgés de 18 à 25 ans
* SNHMH2614 : salaire net moyen par heure pour les hommes âgés de 26 à 50 ans
* SNHMH5014 : salaire net moyen par heure pour les hommes de plus de 50 ans


In [14]:
# Lecture du fichier csv
df3 = pd.read_csv('net_salary_per_town_categories.csv')
# Mise en majuscule de la première lettre du nom de commune LIBGEO
df3.LIBGEO = df3.LIBGEO.str.title()
# Suppression des accents les plus courants sur la majuscule
df3['LIBGEO'] = df3['LIBGEO'].str.replace("É","E")
df3['LIBGEO'] = df3['LIBGEO'].str.replace("Î","I")
# Extraction des variables Geo uniquement
df3_base = df3.iloc[:,0:2]
# Affichage des premières de la DF
df3_base.head()

,CODGEO,LIBGEO
0,01004,Ambérieu-En-Bugey
1,01007,Ambronay
2,01014,Arbent
3,01024,Attignat
4,01025,Bâgé-La-Ville


In [15]:
# affiche de la dimension de la DF
df3_base.shape

(5136, 2)

In [16]:
# Recherche de nan
df3_base.isna().sum()

CODGEO    0
LIBGEO    0
dtype: int64

### Informations démographiques par ville, âge, sexe et mode de vie

<u>population.csv : </u>

* NIVGEO : geographic level (arrondissement, communes…)
* CODGEO : unique code for the town
* LIBGEO : name of the town
* MOCO : mode de cohabitation :
    * 11 = enfants vivant avec deux parents
    * 12 = enfants vivant avec un seul parent
    * 21 = adultes vivant en couple sans enfant
    * 22 = adultes vivant en couple avec enfants
    * 23 = adultes vivant seuls avec enfants
    * 31 = personnes étrangères à la famille vivant au foyer
    * 32 = personnes vivant seules
* AGE80_17 : catégorie d'âge (tranche de 5 ans) | ex : 0 -> personnes âgées de 0 à 4 ans
* SEXE : sexe, 1 pour homme | 2 pour femme
* NB : Nombre de personnes dans la catégorie

In [17]:
# Lecture du fichier de données population
df4 = pd.read_csv('population.csv',low_memory=False)
# Mise en majuscule de la première lettre du nom de commune LIBGEO 
df4.LIBGEO = df4.LIBGEO.str.title()
# Suppression des accents les plus courants sur la majuscule
df4['LIBGEO'] = df4['LIBGEO'].str.replace("É","E")
df4['LIBGEO'] = df4['LIBGEO'].str.replace("Î","I")
# Affichage des infos de la DF
df4.info()
# df_temp = df4.loc[df4.CODGEO=='14472']
# df_temp.CODGEO = '14697'
# df1.loc[df1.CODGEO=='14472']=df_temp

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8536584 entries, 0 to 8536583
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   NIVGEO     object
 1   CODGEO     object
 2   LIBGEO     object
 3   MOCO       int64 
 4   AGEQ80_17  int64 
 5   SEXE       int64 
 6   NB         int64 
dtypes: int64(4), object(3)
memory usage: 455.9+ MB


In [18]:
# On groupe les données par commune pour obtenir la population
# Création d'une nouvelle DF population
population = pd.DataFrame(df4.groupby(['NIVGEO','CODGEO','LIBGEO'])['NB'].sum(),index=None)
# Affichage de la dimension
population.shape

(35868, 1)

In [19]:
# Verification des nan
population.isna().sum()

NB    0
dtype: int64

In [20]:
# Fusion des DF salaires et population
df3_base_pop = pd.merge(population,df3_base,how='outer',on=['CODGEO','LIBGEO'])
# Affichage de la dimension de la DF
df3_base_pop.shape

(35984, 3)

In [21]:
# Vérification des nan
df3_base_pop.isna().sum()

CODGEO      0
LIBGEO      0
NB        116
dtype: int64

In [22]:
df3_base_pop.loc[df3_base_pop.NB.isna()]

,CODGEO,LIBGEO,NB
35868,01426,Treffort-Cuisiat,NaN
35869,10003,Aix-En-Othe,NaN
35870,12270,Sévérac-Le-Château,NaN
35871,14174,Condé-Sur-Noireau,NaN
35872,14371,Livarot,NaN
...,...,...,...
35979,85090,La Flocellière,NaN
35980,85213,Saint-Florent-Des-Bois,NaN
35981,85257,Saint-Michel-Mont-Mercure,NaN
35982,88218,Granges-Sur-Vologne,NaN


In [23]:
# Fusion des DF salaires/population et all_content (entreprises)
all_content_pop = pd.merge(all_content,df3_base_pop,how='outer',on=['CODGEO','LIBGEO'])
# Affichage de la dimension
all_content_pop.shape

(37099, 11)

In [24]:
# Affichage des premières lignes de la nouvelle DF
all_content_pop.head()

,CODGEO,LIBGEO,REG,DEP,nom_région,nom_département,chef.lieu_région,Code_postal,latitude,longitude,NB
0,01001,L'Abergement-Clémenciat,82,01,Rhône-Alpes,Ain,Bourg-en-Bresse,1400.0,46.153721,4.925850,765.0
1,01002,L'Abergement-De-Varey,82,01,Rhône-Alpes,Ain,Bourg-en-Bresse,1640.0,46.009606,5.428088,227.0
2,01004,Ambérieu-En-Bugey,82,01,Rhône-Alpes,Ain,Bourg-en-Bresse,1500.0,45.961049,5.372275,13460.0
3,01005,Ambérieux-En-Dombes,82,01,Rhône-Alpes,Ain,Bourg-en-Bresse,1330.0,45.996164,4.911967,1595.0
4,01006,Ambléon,82,01,Rhône-Alpes,Ain,Bourg-en-Bresse,1300.0,45.749886,5.594585,84.0


In [25]:
# Affichage des nan
all_content_pop.isna().sum()

CODGEO                 0
LIBGEO                 0
REG                  418
DEP                  418
nom_région           418
nom_département      418
chef.lieu_région     418
Code_postal         2157
latitude            2157
longitude           2157
NB                  1231
dtype: int64

In [27]:
# Extraction des CODGEO dupliqués
all_content_pop_dupli = all_content_pop.loc[all_content_pop.duplicated(subset=['CODGEO'])]
# Affichage des lignes dupliquées
all_content_pop_dupli

,CODGEO,LIBGEO,REG,DEP,nom_région,nom_département,chef.lieu_région,Code_postal,latitude,longitude,NB
36681,01015,Arboys En Bugey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,616.0
36682,01080,Champdor-Corcelles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,632.0
36683,01187,Haut Valromey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,717.0
36684,01204,Le Poizat-Lalleyriat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,695.0
36685,01286,Parves Et Nattages,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,868.0
...,...,...,...,...,...,...,...,...,...,...,...
37094,89196,Valravillon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1629.0
37095,89334,Le Val D'Ocre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,562.0
37096,89359,Saint-Maurice-Aux-Riches-Hommes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,298.0
37097,89388,Sépeaux-Saint Romain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,567.0


In [28]:
# Extraction de la population par commune NB
nb = all_content_pop_dupli.NB.values
# Affichage du nombre d'élement
len(nb)

417

In [29]:
# Après vérification de la structure des duplication de la DF, 
# on peut supprimer les lignes dupliquées en ne gardant que la première
all_content_pop = all_content_pop.drop_duplicates(subset=['CODGEO'],keep='first')

In [30]:
# Extraction des CODGEO des lignes dupliquées
codgeo = all_content_pop_dupli.CODGEO.values
# Creation d'une DF temporaire
df_codgeo=pd.DataFrame()
# Pore chaque codegeo dupliqué
for cod in codgeo:
    # extraction des lignes avec le code dupliqué
    df_temp = all_content_pop.loc[all_content_pop.CODGEO==cod]
    # Extraction de la première ligne
    df_temp = df_temp.iloc[0]
    # Ajout la la DF temporaire
    df_codgeo = df_codgeo.append(df_temp)
# On complete l'extrait avec la variable NB (population par commune)
df_codgeo.NB = nb
# Affichage de la DF
df_codgeo

,CODGEO,LIBGEO,REG,DEP,nom_région,nom_département,chef.lieu_région,Code_postal,latitude,longitude,NB
13,01015,Arbignieu,82,01,Rhône-Alpes,Ain,Bourg-en-Bresse,1300.0,45.712797,5.641677,616.0
73,01080,Champdor,82,01,Rhône-Alpes,Ain,Bourg-en-Bresse,1110.0,46.028576,5.588009,632.0
170,01187,Hotonnes,82,01,Rhône-Alpes,Ain,Bourg-en-Bresse,1260.0,46.047458,5.687541,717.0
186,01204,Lalleyriat,82,01,Rhône-Alpes,Ain,Bourg-en-Bresse,1130.0,46.145021,5.695221,695.0
258,01286,Parves,82,01,Rhône-Alpes,Ain,Bourg-en-Bresse,1300.0,45.731043,5.745541,868.0
...,...,...,...,...,...,...,...,...,...,...,...
35671,89196,Guerchy,26,89,Bourgogne,Yonne,Auxerre,89113.0,47.904349,3.431627,1629.0
35802,89334,Saint-Aubin-Château-Neuf,26,89,Bourgogne,Yonne,Auxerre,89110.0,47.819705,3.295130,562.0
35826,89359,Saint-Maurice-Aux-Riches-Hom,26,89,Bourgogne,Yonne,Auxerre,89190.0,48.329645,3.498544,298.0
35854,89388,Sépeaux,26,89,Bourgogne,Yonne,Auxerre,89116.0,47.935991,3.239130,567.0


In [51]:
all_content_pop.NB = all_content_pop.NB.fillna(value=df_codgeo.NB)

In [52]:
# # Extraction des lignes dupliquées
all_content_pop_dupli = all_content_pop.loc[all_content_pop.duplicated(subset=['CODGEO'])]
# # Affichage des lignes dupliquée
all_content_pop_dupli

,CODGEO,LIBGEO,REG,DEP,nom_région,nom_département,chef.lieu_région,Code_postal,latitude,longitude,NB


In [49]:
# Affichage de la dimension de la DF
all_content_pop.shape

(36682, 11)

In [53]:
# CODGEO : ID géographique de la ville
# LIBGEO : nom de la ville
# REG : numéro de région
# DEP : numéro de département
# E14TST : nombre total d'entreprises dans la ville
# E14TS0ND : nombre d'entreprises de taille inconnue ou nulle dans la ville
# E14TS1 : nombre d'entreprises de 1 à 5 employés dans la ville
# E14TS6 : nombre d'entreprises de 6 à 9 employés dans la ville
# E14TS10 : nombre d'entreprises de 10 à 19 employés dans la ville
# E14TS20 : nombre d'entreprises de 20 à 49 employés dans la ville
# E14TS50 : nombre d'entreprises de 50 à 99 employés dans la ville
# E14TS100 : nombre d'entreprises de 100 à 199 employés dans la ville
# E14TS200 : nombre d'entreprises de 200 à 499 employés dans la ville
# E14TS500 : nombre d'entreprises de plus de 500 employés dans la ville
df1_E14T = df1.drop(labels=['LIBGEO','REG','DEP'],axis='columns')
df1_E14T

,CODGEO,E14TST,E14TS0ND,E14TS1,E14TS6,E14TS10,E14TS20,E14TS50,E14TS100,E14TS200,E14TS500
0,01001,25,22,1,2,0,0,0,0,0,0
1,01002,10,9,1,0,0,0,0,0,0,0
2,01004,996,577,272,63,46,24,9,3,2,0
3,01005,99,73,20,3,1,2,0,0,0,0
4,01006,4,4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
36676,97613,169,169,0,0,0,0,0,0,0,0
36677,97614,230,230,0,0,0,0,0,0,0,0
36678,97615,569,569,0,0,0,0,0,0,0,0
36679,97616,345,345,0,0,0,0,0,0,0,0


In [54]:
# Fusion de all content pop avec les données des entreprises 
all_content_pop_entr = pd.merge(all_content_pop,df1_E14T,how='right',on='CODGEO')
all_content_pop_entr.shape

(36681, 21)

In [55]:
all_content_pop_entr.isna().sum()

CODGEO                 0
LIBGEO                 0
REG                    0
DEP                    0
nom_région             0
nom_département        0
chef.lieu_région       0
Code_postal         1739
latitude            1739
longitude           1739
NB                   814
E14TST                 0
E14TS0ND               0
E14TS1                 0
E14TS6                 0
E14TS10                0
E14TS20                0
E14TS50                0
E14TS100               0
E14TS200               0
E14TS500               0
dtype: int64

In [56]:
all_content_pop_entr.loc[all_content_pop_entr.E14TST.isna()]

,CODGEO,LIBGEO,REG,DEP,nom_région,nom_département,chef.lieu_région,Code_postal,latitude,longitude,...,E14TST,E14TS0ND,E14TS1,E14TS6,E14TS10,E14TS20,E14TS50,E14TS100,E14TS200,E14TS500


In [57]:
# CODGEO : ID géographique de la ville
# LIBGEO : nom de la ville
# SNHM14 : salaire net moyen
# SNHMC14 : salaire net moyen par heure pour les cadres
# SNHMP14 : salaire net moyen par heure pour un cadre moyen
# SNHME14 : salaire net moyen par heure pour l'employé
# SNHMO14 : salaire net moyen par heure pour le travailleur
# SNHMF14 : salaire net moyen pour les femmes
# SNHMFC14 : salaire net moyen par heure pour les cadres féminins
# SNHMFP14 : salaire net moyen par heure pour les cadres moyens féminins
# SNHMFE14 : salaire net moyen par heure pour une employée
# SNHMFO14 : salaire net moyen par heure pour une travailleuse
# SNHMH14 : salaire net moyen pour un homme
# SNHMHC14 : salaire net moyen par heure pour un cadre masculin
# SNHMHP14 : salaire net moyen par heure pour les cadres moyens masculins
# SNHMHE14 : salaire net moyen par heure pour un employé masculin
# SNHMHO14 : salaire net moyen par heure pour un travailleur masculin
# SNHM1814 : salaire net moyen par heure pour les 18-25 ans
# SNHM2614 : salaire net moyen par heure pour les 26-50 ans
# SNHM5014 : salaire net moyen par heure pour les >50 ans
# SNHMF1814 : salaire net moyen par heure pour les femmes âgées de 18 à 25 ans
# SNHMF2614 : salaire net moyen par heure pour les femmes âgées de 26 à 50 ans
# SNHMF5014 : salaire net moyen par heure pour les femmes de plus de 50 ans
# SNHMH1814 : salaire net moyen par heure pour les hommes âgés de 18 à 25 ans
# SNHMH2614 : salaire net moyen par heure pour les hommes âgés de 26 à 50 ans
# SNHMH5014 : salaire net moyen par heure pour les hommes de plus de 50 ans
df3_SNHM14 = df3.drop(labels='LIBGEO',axis='columns')
df3_SNHM14

,CODGEO,SNHM14,SNHMC14,SNHMP14,SNHME14,SNHMO14,SNHMF14,SNHMFC14,SNHMFP14,SNHMFE14,...,SNHMHO14,SNHM1814,SNHM2614,SNHM5014,SNHMF1814,SNHMF2614,SNHMF5014,SNHMH1814,SNHMH2614,SNHMH5014
0,01004,13.7,24.2,15.5,10.3,11.2,11.6,19.1,13.2,10.1,...,11.6,10.5,13.7,16.1,9.7,11.8,12.5,11.0,14.9,18.6
1,01007,13.5,22.1,14.7,10.7,11.4,11.9,19.0,13.3,10.6,...,11.7,9.8,13.8,14.6,9.2,12.2,12.5,10.2,14.9,16.4
2,01014,13.5,27.6,15.6,11.1,11.1,10.9,19.5,11.7,10.8,...,11.8,9.3,13.3,16.0,8.9,10.6,12.5,9.6,15.1,18.6
3,01024,12.9,21.8,14.1,11.0,11.3,11.4,19.0,13.0,10.3,...,11.6,9.6,12.9,14.2,9.3,11.4,12.2,9.7,13.8,15.9
4,01025,13.0,22.8,14.1,10.5,11.1,11.6,19.4,13.6,10.2,...,11.4,9.4,12.8,15.2,9.0,11.8,12.3,9.7,13.4,16.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,97420,12.9,24.5,15.4,10.9,10.9,12.4,21.6,14.7,10.8,...,11.1,9.2,12.8,15.4,9.3,12.3,14.7,9.2,13.1,15.8
5132,97421,10.4,17.3,13.8,9.6,9.8,9.8,13.2,11.4,9.6,...,10.0,9.0,10.6,11.4,8.9,9.9,10.5,9.0,11.0,11.7
5133,97422,12.0,23.3,14.7,10.3,10.5,11.5,21.3,14.1,10.2,...,10.6,8.9,11.9,14.9,8.9,11.4,13.9,9.0,12.1,15.4
5134,97423,11.4,22.6,13.4,10.1,10.5,10.8,15.9,12.4,10.1,...,10.5,8.9,11.5,13.1,8.9,11.1,11.4,9.0,11.8,13.7


In [58]:
# Fusion de all content pop entreprises avec les données de salaire
all_content_2014 = pd.merge(all_content_pop_entr,df3_SNHM14,how='left',on='CODGEO')
all_content_2014.shape

(36681, 45)

In [47]:
all_content_2014.isna().sum()

CODGEO                  0
LIBGEO                  0
REG                     0
DEP                     0
nom_région              0
nom_département         0
chef.lieu_région        0
Code_postal          1739
latitude             1739
longitude            1739
NB                    814
E14TST                  0
E14TS0ND                0
E14TS1                  0
E14TS6                  0
E14TS10                 0
E14TS20                 0
E14TS50                 0
E14TS100                0
E14TS200                0
E14TS500                0
SNHM14              31545
SNHMC14             31545
SNHMP14             31545
SNHME14             31545
SNHMO14             31545
SNHMF14             31545
SNHMFC14            31545
SNHMFP14            31545
SNHMFE14            31545
SNHMFO14            31545
SNHMH14             31545
SNHMHC14            31545
SNHMHP14            31545
SNHMHE14            31545
SNHMHO14            31545
SNHM1814            31545
SNHM2614            31545
SNHM5014    

In [59]:
# Sauvegarde de la DF all_content_pop dans un fichier csv
all_content_2014.to_csv("all_content_2014_v2.csv",encoding= "utf-8")